In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

/home/carlos/miniconda3/envs/Genome-Transition/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('../../data/data_ie.csv')
df.head(), df.shape

(             gene_id  chromosome  global_position  Exon_Start B1 B2 B3 B4 B5  \
 0  ENSG00000260861.6          20          1556582       18437  t  t  t  t  a   
 1  ENSG00000260861.6          20          1610358       72213  a  g  c  t  g   
 2  ENSG00000260861.6          20          1577339       39194  a  t  t  g  a   
 3  ENSG00000260861.6          20          1587561       49416  g  c  c  c  a   
 4  ENSG00000260861.6          20          1577339       39194  a  t  t  g  a   
 
   B6  ... B97 B98 B99 B100 B101 B102 B103 B104 B105 label  
 0  c  ...   a   a   a    a    t    g    g    a    g  True  
 1  a  ...   t   a   g    c    a    a    c    t    t  True  
 2  t  ...   c   a   t    c    a    g    g    g    t  True  
 3  c  ...   g   c   a    c    c    t    a    g    g  True  
 4  t  ...   c   a   t    c    a    g    g    g    t  True  
 
 [5 rows x 110 columns],
 (22391, 110))

In [5]:
seq_col = [c for c in df.columns if c.startswith('B')]
df_model = df[seq_col + ['label']]

df_model['label'] = (
    df_model['label'].astype(str).str.lower().map({"true":1, "false":0})
)

df_model['label'].value_counts(dropna=False)

df_model.head()

train_val,test_data = train_test_split(
    df_model, 
    test_size=0.2, 
    random_state=42,
    stratify=df_model['label']
)

train , val = train_test_split(
    train_val,
    test_size=0.25,
    random_state=42,
    stratify=train_val['label']
)

train_val.shape, train.shape, val.shape, test_data.shape





/tmp/ipykernel_253022/3480706333.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['label'] = (


((17912, 106), (13434, 106), (4478, 106), (4479, 106))

In [6]:
predictor = TabularPredictor(
    label="label", 
    problem_type="binary",
    eval_metric="f1",
    path="../models/autogluon_ie")


predictor.fit(
    train_data=train,
    tuning_data=val,
    time_limit=200,
    presets="medium_quality_faster_train",
)

Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
CPU Count:          12
Pytorch Version:    2.6.0
CUDA Version:       12.6
GPU Memory:         GPU 0: 6.00/6.00 GB
Total GPU Memory:   Free: 6.00 GB, Allocated: 0.00 GB, Total: 6.00 GB
GPU Count:          1
Memory Avail:       8.30 GB / 11.55 GB (71.8%)
Disk Space Avail:   640.18 GB / 951.65 GB (67.3%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 200s
AutoGluon will save models to "/mnt/c/Users/Carlos/OneDrive/Documentos/Genome-Transition_AutoGluon/training/models/autogluon_ie"
Train Data Rows:    13434
Train Data Columns: 105
Tuning Data Rows:

In [7]:
perf = predictor.evaluate(test_data)
perf

{'f1': 0.9964216900633086,
 'accuracy': 0.9970975664210762,
 'balanced_accuracy': np.float64(0.9968602835480842),
 'mcc': 0.9939813373628438,
 'roc_auc': np.float64(0.9998569567591858),
 'precision': 0.9972451790633609,
 'recall': 0.9955995599559956}

In [8]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.996426,0.995033,f1,0.141720,0.058163,127.779429,0.141720,0.058163,127.779429,1,True,5
1,XGBoost,0.996422,0.996137,f1,0.149185,0.098860,10.244052,0.149185,0.098860,10.244052,1,True,9
2,WeightedEnsemble_L2,0.996422,0.996137,f1,0.211178,0.101589,10.878939,0.061993,0.002729,0.634887,2,True,10
3,LightGBM,0.994509,0.995308,f1,0.077521,0.038307,2.664384,0.077521,0.038307,2.664384,1,True,2
4,LightGBMXT,0.993953,0.995028,f1,0.118686,0.041134,3.850171,0.118686,0.041134,3.850171,1,True,1
5,RandomForestGini,0.992877,0.992252,f1,0.673982,0.153607,1.514847,0.673982,0.153607,1.514847,1,True,3
6,ExtraTreesEntr,0.991785,0.993634,f1,0.769215,0.165176,1.274482,0.769215,0.165176,1.274482,1,True,7
7,RandomForestEntr,0.991772,0.991690,f1,0.459783,0.152923,1.569029,0.459783,0.152923,1.569029,1,True,4
8,ExtraTreesGini,0.990164,0.993913,f1,0.736846,0.164521,1.350826,0.736846,0.164521,1.350826,1,True,6
9,NeuralNetFastAI,0.986490,0.988926,f1,1.476318,1.707838,43.266718,1.476318,1.707838,43.266718,1,True,8


In [9]:
predictor.feature_importance(test_data).head(10)

Computing feature importance via permutation shuffling for 105 features using 4479 rows with 5 shuffle sets...
	315.41s	= Expected runtime (63.08s per shuffle set)
	68.11s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
B100,0.227362,0.002343,1.352944e-09,5,0.232186,0.222538
B99,0.206876,0.005275,5.068530e-08,5,0.217737,0.196014
B98,0.015834,0.001857,2.231415e-05,5,0.019658,0.012009
B96,0.002047,0.000373,1.270015e-04,5,0.002816,0.001279
B90,0.001381,0.000435,1.041601e-03,5,0.002278,0.000485
B101,0.001163,0.000411,1.594122e-03,5,0.002010,0.000317
B87,0.001161,0.000303,5.107149e-04,5,0.001786,0.000537
B83,0.001161,0.000660,8.531967e-03,5,0.002520,-0.000198
B95,0.000996,0.000247,4.209486e-04,5,0.001505,0.000487
B93,0.000885,0.000231,5.129300e-04,5,0.001361,0.000409
